## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-19-23-48-52 +0000,bbc,"White House has apologised over Georgia raid, ...",https://www.bbc.com/news/articles/c8ey0nl62ypo...
1,2025-11-19-23-44-25 +0000,nypost,House unanimously passes bill to crack down on...,https://nypost.com/2025/11/19/us-news/house-un...
2,2025-11-19-23-43-09 +0000,nypost,Lindsey Halligan fires back at James Comey jud...,https://nypost.com/2025/11/19/us-news/lindsey-...
3,2025-11-19-23-42-32 +0000,nypost,"Trump, Elon Musk bromance back on: Inside how ...",https://nypost.com/2025/11/19/us-news/bromance...
4,2025-11-19-23-40-55 +0000,nypost,MTA drops $257M on new bus fleet in latest no-...,https://nypost.com/2025/11/19/us-news/mta-drop...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,39
484,epstein,23
36,new,21
64,saudi,18
80,ukraine,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2025-11-19-15-01-08 +0000,nyt,Here’s Who’s Attending Trump’s Dinner for the ...,https://www.nytimes.com/2025/11/18/us/politics...,98
9,2025-11-19-23-27-18 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...,97
79,2025-11-19-21-17-07 +0000,nyt,Trump Says He Will Approve the Epstein Files R...,https://www.nytimes.com/2025/11/19/us/politics...,96
26,2025-11-19-22-51-00 +0000,wsj,Trump Administration Pushes New Plan for Endin...,https://www.wsj.com/world/trump-administration...,94
76,2025-11-19-21-23-35 +0000,nypost,Grocery prices to remain high through Thanksgi...,https://nypost.com/2025/11/19/business/grocery...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,98,2025-11-19-15-01-08 +0000,nyt,Here’s Who’s Attending Trump’s Dinner for the ...,https://www.nytimes.com/2025/11/18/us/politics...
77,81,2025-11-19-21-19-29 +0000,nypost,Ex-Harvard Prez Larry Summers tells students h...,https://nypost.com/2025/11/19/us-news/ex-harva...
11,59,2025-11-19-23-21-58 +0000,nypost,New England serial killer with haunting past r...,https://nypost.com/2025/11/19/us-news/new-engl...
56,53,2025-11-19-21-52-57 +0000,nypost,‘Off the charts’ demand for AI chips powers st...,https://nypost.com/2025/11/19/business/nvidia-...
257,52,2025-11-19-11-26-33 +0000,nypost,White House rips House Democrat’s ‘sick’ defen...,https://nypost.com/2025/11/19/us-news/white-ho...
65,52,2025-11-19-21-39-38 +0000,bbc,Children among 26 killed in one of Russia's de...,https://www.bbc.com/news/articles/cy0yere4k0zo...
174,43,2025-11-19-16-32-56 +0000,nypost,"Trump Media stock crashes to all-time lows, wi...",https://nypost.com/2025/11/19/business/trump-m...
94,38,2025-11-19-20-41-11 +0000,nyt,Israeli Strikes Kill at Least 25 in Gaza Amid ...,https://www.nytimes.com/2025/11/19/world/middl...
95,37,2025-11-19-20-40-00 +0000,latimes,Olympic snowboarder turned drug lord ordered k...,https://www.latimes.com/california/story/2025-...
25,35,2025-11-19-22-52-03 +0000,nyt,U.S.-Russian Peace Plan Would Force Ukraine to...,https://www.nytimes.com/2025/11/19/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
